
**Why Vertex AI Workbench Doesn't Find PySpark by Default**

  * **JupyterLab Environment:** A Vertex AI Workbench instance provides a JupyterLab environment running on a Compute Engine VM. While it comes with many data science libraries (TensorFlow, PyTorch, scikit-learn), it **doesn't inherently include a Spark distribution or PySpark configured to run locally or connect to a Spark cluster.**
  * **Spark is a Distributed System:** PySpark is the Python API for Apache Spark, which is a *distributed computing framework*. It's designed to run across a cluster of machines. A single notebook instance, by itself, isn't a Spark cluster.
  * **Missing Dependencies:** PySpark requires a Java Development Kit (JDK) and specific environment variables (`SPARK_HOME`, `JAVA_HOME`) to be set up correctly to find and interact with the underlying Spark binaries.

**How to Enable PySpark in Vertex AI Workbench**

There are two primary ways to get PySpark working in a Vertex AI Workbench notebook:

**Method 1: Integrate with a Dataproc Cluster (Recommended for Big Data)**

This is the most robust and scalable solution for real Big Data workloads. You connect your Workbench notebook to a managed Spark cluster (Dataproc).

1.  **Enable Dataproc API:** Ensure the Dataproc API is enabled in your Google Cloud project.
    ```bash
    gcloud services enable dataproc.googleapis.com
    ```
2.  **Create a Dataproc Cluster:** Create a Dataproc cluster in the same region as your Workbench instance. You can do this via the GCP Console or `gcloud` commands.
    ```bash

    gcloud dataproc clusters create example-cluster\
      --enable-component-gateway\
      --bucket=example-dataproc-workshop\
      --region=europe-west1\
      --no-address\
      --master-machine-type=n4-standard-2\
      --master-boot-disk-type=hyperdisk-balanced\
      --master-boot-disk-size=100\
      --num-workers=2\
      --worker-machine-type=n4-standard-2\
      --worker-boot-disk-size=200\
      --image-version=2.2-debian12\
      --optional-components JUPYTER\
      --max-age=3600s\
      --labels=mode=workshop,user=zelda\
      --project=$PROJECT_ID
    ```
3.  **Enable Dataproc Integration in Workbench:**
      * When creating a new Vertex AI Workbench instance, ensure "Enable Dataproc Serverless Interactive Sessions" (or "Enable Dataproc" for older versions) is selected under "Advanced options" -\> "Environment" or "Integrations."
      * If your instance already exists, you might need to stop it, edit it, enable the Dataproc integration, and then restart it.
4.  **Open JupyterLab and Select Dataproc Kernel:**
      * Once your Workbench instance is running and has Dataproc integration enabled, open JupyterLab.
      * When creating a new notebook, you should see kernels like "PySpark" or "Spark (with Python 3)" that allow you to connect to your Dataproc cluster.
5.  **Connect to Spark Session in Notebook:** Your notebook code will then automatically connect to the Dataproc cluster when you create a SparkSession.
    ```python
    from pyspark.sql import SparkSession

    spark = SparkSession.builder.appName("MyPySparkApp").getOrCreate()
    print("SparkSession created successfully!")
    ```

**Method 2: Install PySpark Locally on the Workbench Instance (for Smaller Scale/Testing)**

This method installs PySpark directly on your Workbench VM. It's suitable for smaller datasets that fit within the VM's memory and for local development/testing, but it won't leverage distributed computing.

1.  **Open a Terminal in JupyterLab:** From your JupyterLab interface on the Workbench instance, go to `File` -\> `New` -\> `Terminal`.
2.  **Install Java (if not present):** PySpark requires Java. Many default images have it, but if not:
    ```bash
    sudo apt-get update
    sudo apt-get install default-jdk
    ```
3.  **Install PySpark via pip:**
    ```bash
    pip install pyspark
    ```
4.  **Set Environment Variables (Optional, but good practice):**
    ```bash
    # Add these to your ~/.bashrc or directly in the notebook cell
    export JAVA_HOME="/usr/lib/jvm/default-java" # Adjust if your JDK path is different
    export SPARK_HOME="/opt/conda/lib/python3.11/site-packages/pyspark" # Adjust based on your pip install location
    export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
    ```
      * You might need to restart the kernel or JupyterLab after setting these.
5.  **Create SparkSession in Notebook:**
    ```python
    from pyspark.sql import SparkSession

    spark = SparkSession.builder.appName("LocalPySpark").getOrCreate()
    print("Local SparkSession created successfully!")
    ```

**Troubleshooting Tips:**

  * **Check Java Installation:** Run `java -version` in the terminal to confirm Java is installed.
  * **Environment Variables:** Ensure `JAVA_HOME` and `SPARK_HOME` are correctly set.
  * **Kernel Selection:** Always choose the correct PySpark kernel in your JupyterLab notebook.
  * **Restart Kernel:** After installing packages or changing environment variables, always restart the notebook kernel.


# Demo

Start by creating the cluster, you can launch this command here (use Python 3 kernel) or in a normal shell.

In [1]:
!gcloud dataproc clusters create example-cluster\
      --enable-component-gateway\
      --bucket=example-dataproc-workshop\
      --region=europe-west1\
      --no-address\
      --master-machine-type=n4-standard-2\
      --master-boot-disk-type=hyperdisk-balanced\
      --master-boot-disk-size=100\
      --num-workers=2\
      --worker-machine-type=n4-standard-2\
      --worker-boot-disk-size=200\
      --image-version=2.2-debian12\
      --optional-components JUPYTER\
      --max-age=3600s\
      --labels=mode=workshop,user=zelda\
      --project=poc-example-ds

Waiting on operation [projects/poc-example-ds/regions/europe-west1/operations/ca1b0bbe-23d8-3def-86f8-4918d4fbf636].
Waiting for cluster creation operation...                                      
Waiting for cluster creation operation...done.                                 
Created [https://dataproc.googleapis.com/v1/projects/poc-example-ds/regions/europe-west1/clusters/example-cluster] Cluster placed in zone [europe-west1-b].


Now select the kernel again (upper right corner) and set it to pyspark "CLUSTER_NAME" in our case example-cluster. 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Get or create a SparkSession.
spark = SparkSession.builder \
    .appName("BasicPySparkOperations") \
    .getOrCreate()

print("SparkSession created successfully!")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")
print(f"Spark Version: {spark.version}")

SparkSession created successfully!
Spark UI: http://example-cluster-m.europe-west1-b.c.poc-example-ds.internal:39695
Spark Version: 3.5.3


25/07/28 14:24:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# process_data.py
import argparse
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType



def main(input_path, processing_date, output_table):

 

    spark = SparkSession.builder \
        .appName("SalesDataProcessor") \
        .config("temporaryGcsBucket", "example-dataproc-workshop") \
        .getOrCreate()

    # Define schema for input data (adjust as per your actual data)
    input_schema = StructType([
        StructField("transaction_id", StringType(), True),
        StructField("product_id", StringType(), True),
        StructField("amount", DoubleType(), True),
        StructField("transaction_date", DateType(), True)
    ])

    try:
        print(f"Reading data from: {input_path}")
        df = spark.read \
            .option("header", "true") \
            .schema(input_schema) \
            .csv(input_path)

        print(f"Processing data for date: {processing_date}")
        # Example processing: Filter by date and calculate total sales
        processed_df = df.filter(F.col("transaction_date") == F.lit(processing_date)) \
                         .groupBy("product_id") \
                         .agg(F.sum("amount").alias("total_sales")) \
                         .withColumn("processing_date", F.lit(processing_date))

        # Example of using a utility function
        # processed_df = some_utility_function(processed_df)

        print(f"Writing processed data to BigQuery table: {output_table}")


        # Write to BigQuery
        processed_df.write \
            .format("bigquery") \
            .option("table", output_table) \
            .mode("append").save()
        # mode can be  "append" or "overwrite" or "ignore" or "errorifexists"
        print("PySpark job completed successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
        spark.stop()
        raise # Re-raise the exception to indicate job failure

    finally:
        spark.stop()

if __name__ == "__main__":
    main(input_path="gs://example-dataproc-workshop/01_2_Submit_Job/raw_data/sales/",
     output_table="example_dataproc_workshop.processed_sales",
    processing_date="2025-06-12")

25/07/28 14:24:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Reading data from: gs://example-dataproc-workshop/01_2_Submit_Job/raw_data/sales/
Processing data for date: 2025-06-12
Writing processed data to BigQuery table: example_dataproc_workshop.processed_sales


25/07/28 14:24:51 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/07/28 14:25:06 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/07/28 14:25:21 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/07/28 14:25:36 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/07/28 14:25:51 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/07/28 14:26:06 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registere